In [1]:
!pip install accelerate datasets transformers evaluate tensorboard > None

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import evaluate
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, load_metric

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

np.random.seed(42)

In [8]:
print(transformers.__version__)

4.34.0


In [30]:
from google.colab import drive
drive.mount("/content/d")

df = pd.read_csv(
    "d/MyDrive/iftm_nlp/data/women_clothing_sentiment.csv",
    header=0
)
df.head(3)

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative


In [31]:
df.sentiment.value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [34]:
label2id = {label:i for i,label in enumerate(df.sentiment.unique())}
id2label = {i:label for i,label in enumerate(df.sentiment.unique())}
print(label2id,"\n",id2label)

{'negative': 0, 'neautral': 1, 'positive': 2} 
 {0: 'negative', 1: 'neautral', 2: 'positive'}


In [35]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [50]:
model_name = "d0rj/e5-small-en-ru"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    max_length=128,
    padding=True,
    # truncate=True,
    truncation=True,
    return_tensors="pt"
)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fcb81083-3d4d-49a7-a895-ab93e59a7ed3)')' thrown while requesting HEAD https://huggingface.co/d0rj/e5-small-en-ru/resolve/main/tokenizer_config.json


In [51]:
len(tokenizer(df.iloc[8].review)["input_ids"]), tokenizer(df.iloc[8].review)

(190,
 {'input_ids': [0, 676, 7655, 638, 52727, 48, 6, 2750, 937, 5, 496, 6169, 8399, 124, 14174, 362, 173, 32438, 17959, 4161, 238, 55367, 129, 57855, 127, 41476, 6, 36785, 8882, 361, 2956, 2575, 53749, 86, 231, 235, 28859, 4762, 173, 45470, 790, 6103, 540, 21342, 875, 49247, 531, 26975, 875, 21100, 124, 14174, 740, 5, 56231, 4161, 129, 8220, 74, 9486, 13941, 339, 4, 41305, 848, 25766, 465, 32001, 36884, 55741, 4, 634, 74, 32790, 3373, 5, 38802, 74, 48, 5354, 715, 4, 2452, 11000, 2175, 5, 3048, 17618, 20343, 7083, 238, 16094, 608, 36584, 5, 62, 2590, 6956, 84, 4267, 23, 10113, 5, 26698, 67, 36018, 434, 13478, 271, 46, 67, 44169, 560, 21068, 127, 5310, 13, 80, 10, 159, 53112, 19, 394, 387, 25, 7, 28875, 231, 130, 67, 4478, 42, 43112, 68, 46, 42950, 67, 31627, 580, 399, 9, 17615, 67, 45915, 5, 84, 10860, 271, 46, 2639, 11715, 4, 67, 2920, 1296, 37245, 68, 46, 26380, 20878, 4, 999, 8265, 25, 18, 5, 1235, 25, 7, 765, 67, 18293, 4, 1242, 67, 11936, 128, 10719, 5, 2753, 67, 8567, 26550, 84,

In [52]:
check = tokenizer(df.iloc[8].review)
print(type(check))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [43]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    label2id=label2id,
    id2label=id2label
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at d0rj/e5-small-en-ru and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(60302, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384, padding_idx=0)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=384, out_features=384, 

In [46]:
for name, param in model.named_parameters():
    if name.startswith("classifier"):
        print(name)
        continue
    param.requires_grad = False

classifier.dense.weight
classifier.dense.bias
classifier.out_proj.weight
classifier.out_proj.bias


In [54]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, label2id):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer
        self.label2id = label2id

    def __len__(self):

        return len(self.df)

    def __getitem__(self, idx):
        label = self.df["sentiment"].iloc[idx]
        text = self.df["review"].iloc[idx]
        output = self.tokenizer(
            text,
            max_length=128,
            padding=True,
            truncation=True
        )
        output["label"] = self.label2id[label]

        return output

In [55]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

In [60]:
train_dataloader = DataLoader(
    CustomDataset(
        df=train_df,
        tokenizer=tokenizer,
        label2id=label2id
    ),
    collate_fn=data_collator,
    batch_size=2,
    shuffle=True
)

test_dataloader = DataLoader(
    CustomDataset(
        df=test_df,
        tokenizer=tokenizer,
        label2id=label2id
    ),
    collate_fn=data_collator,
    batch_size=2,
    shuffle=True
)

next(iter(test_dataloader))

{'input_ids': tensor([[    0,   676, 30715,    48, 55700, 20824,   498, 10943,   468,   112,
         23866,     4,  9273,   668, 11314, 50476,   464,     4, 35930,   362,
         37557,  4161,  3074,   567,    48, 24914,     5,     5,     2],
        [    0,   854,  5449, 49501,   180,  6401, 35339,   448,     4, 51537,
         15330,  8895,  1262,  8838,     5,   676, 44971, 34536, 28859,  4888,
             2,     1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0]]), 'labels': tensor([0, 2])}

In [61]:
train_dataloader = CustomDataset(
    df=train_df,
    tokenizer=tokenizer,
    label2id=label2id
)

test_dataloader = CustomDataset(
    df=test_df,
    tokenizer=tokenizer,
    label2id=label2id
)

In [63]:
f1_metric = evaluate.load("f1")
results = f1_metric.compute(
    predictions=[0,1,2,1],
    references=[0,1,1,1],
    average="weighted"
)
print(results)

{'f1': 0.8500000000000001}


In [64]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(
        predictions=predictions,
        references=labels,
        average="weighted"
    )

In [65]:
!mkdir checkpoints

In [66]:
training_args = TrainingArguments(
    output_dir="checkpoints/",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_accumulation_steps=2,
    learning_rate=5e-3,
    num_train_epochs=3,
    logging_strategy="steps",
    logging_steps=20,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="tensorboard"
)

In [67]:
trainer = Trainer(
    model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=test_dataloader,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [ ]:
best_model = trainer.train()

Step,Training Loss,Validation Loss
